In [20]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

In [21]:
from PIL import Image

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [24]:
# Paths
data_dir = '/content/drive/MyDrive/Dataset'  # change if needed
batch_size = 32
num_epochs = 50
img_size = 224

In [25]:
# Data transforms
transform = {
    'train': transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [26]:
# Datasets and loaders
dataset = datasets.ImageFolder(data_dir, transform=transform['train'])
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
val_dataset.dataset.transform = transform['val']

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [27]:
# Load pretrained ResNet-50
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))  # Output layer for your classes
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 190MB/s]


In [28]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [29]:
# Training loop
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / train_size
    epoch_acc = running_corrects.double() / train_size
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    # Validation
    model.eval()
    val_corrects = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = val_corrects.double() / val_size
    print(f"Val Accuracy: {val_acc:.4f}")


Epoch 1/50
Train Loss: 2.2638 Acc: 0.4031
Val Accuracy: 0.6284

Epoch 2/50
Train Loss: 0.7466 Acc: 0.8330
Val Accuracy: 0.7257

Epoch 3/50
Train Loss: 0.2071 Acc: 0.9727
Val Accuracy: 0.8093

Epoch 4/50
Train Loss: 0.0697 Acc: 0.9932
Val Accuracy: 0.8307

Epoch 5/50
Train Loss: 0.0310 Acc: 0.9981
Val Accuracy: 0.8171

Epoch 6/50
Train Loss: 0.0208 Acc: 0.9995
Val Accuracy: 0.8249

Epoch 7/50
Train Loss: 0.0334 Acc: 0.9951
Val Accuracy: 0.7568

Epoch 8/50
Train Loss: 0.0562 Acc: 0.9903
Val Accuracy: 0.7957

Epoch 9/50
Train Loss: 0.0357 Acc: 0.9946
Val Accuracy: 0.7607

Epoch 10/50
Train Loss: 0.0493 Acc: 0.9888
Val Accuracy: 0.7607

Epoch 11/50
Train Loss: 0.0672 Acc: 0.9864
Val Accuracy: 0.7860

Epoch 12/50
Train Loss: 0.0516 Acc: 0.9869
Val Accuracy: 0.7724

Epoch 13/50
Train Loss: 0.0641 Acc: 0.9839
Val Accuracy: 0.7821

Epoch 14/50
Train Loss: 0.1217 Acc: 0.9669
Val Accuracy: 0.7802

Epoch 15/50
Train Loss: 0.0431 Acc: 0.9893
Val Accuracy: 0.8093

Epoch 16/50
Train Loss: 0.0295 Ac

In [30]:
# Evaluation report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=dataset.classes))


Classification Report:
                    precision    recall  f1-score   support

      Akshay Kumar       0.77      0.91      0.83        11
Alexandra Daddario       0.68      1.00      0.81        17
        Alia Bhatt       0.80      0.44      0.57        18
  Amitabh Bachchan       1.00      0.93      0.97        15
      Andy Samberg       0.57      0.94      0.71        18
    Anushka Sharma       0.76      0.87      0.81        15
     Billie Eilish       0.86      1.00      0.93        19
         Brad Pitt       0.87      0.81      0.84        16
    Camila Cabello       0.88      0.91      0.89        23
   Charlize Theron       0.54      0.64      0.58        11
       Claire Holt       0.83      0.59      0.69        17
      Courtney Cox       0.91      0.77      0.83        13
    Dwayne Johnson       1.00      0.89      0.94         9
   Elizabeth Olsen       0.89      0.67      0.76        12
   Ellen Degeneres       0.95      0.95      0.95        19
      Henry Cav

In [31]:
# Save the trained model
save_path = '/content/drive/MyDrive/resnet50_finetuned_2.pth'
torch.save(model.state_dict(), save_path)